In [1]:
import pandas as pd
import os
import gc
from tqdm import tqdm
import numpy as np
import glob
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator
os.getcwd()

'C:\\Users\\Kotani Lab\\Desktop\\ML_senior_project\\ML-Based-Adaptive-Cybersecurity-Incident-Detection\\Code_and_model\\Program'

In [2]:
kdd1 = pd.read_csv('kdd/Result/kdd1/attack_prediction_kdd.csv')
kdd2 = pd.read_csv('kdd/Result/kdd2/attack_prediction_kdd.csv')
cic1 = pd.read_csv('cic/Result/cic1/attack_prediction_cic.csv')
cic2 = pd.read_csv('cic/Result/cic2/attack_prediction_cic.csv')

In [3]:
y_pred1 = kdd1.Prediction
y_pred2 = kdd2.Prediction
y_pred3 = cic1.Prediction
y_pred4 = cic2.Prediction

In [6]:
df_test_kdd = pd.read_csv(glob.glob(f'kdd/train_test_folder/test_kdd/*')[0])
df_test_cic = pd.read_csv(glob.glob(f'cic/train_test_folder/test_cic/*')[0])

test_labels_kdd = df_test_kdd['label']
test_labels_cic = df_test_cic['label']


In [ ]:
def calculate_adaptive(test_labels, y_detect_bi, data_template):
    # print(f'test_labels1 : {test_labels}')
    for i, j in enumerate(y_detect_bi):
        if j == 0:
            if data_template == 'cic':
                test_labels.iloc[i] = 'BENIGN'
            else:
                test_labels.iloc[i] = 'normal'
    return test_labels
    

In [14]:
def convert_labels(test_labels, y_pred, template):
    # Initialize an empty list to store the converted labels
    converted_labels = []
    
    # Iterate through both the test labels and predictions
    for label, pred in zip(test_labels, y_pred):
        if pred == 0 and template == 'kdd':
            converted_labels.append('normal')
        elif pred == 0 and template == 'cic':
            converted_labels.append('BENIGN')
        else:
            converted_labels.append(label)
    
    # Create a new DataFrame from the converted labels
    converted_df = pd.DataFrame(converted_labels, columns=['label'])
    
    return converted_df

In [15]:
converted_df1 = convert_labels(test_labels_kdd, y_pred1, 'kdd')
converted_df2 = convert_labels(test_labels_kdd, y_pred2, 'kdd')
converted_df3 = convert_labels(test_labels_cic, y_pred3, 'cic')
converted_df4 = convert_labels(test_labels_cic, y_pred4, 'cic')


In [24]:
print(f'KDD1 : \n{converted_df1.label.value_counts()}')
print(f'KDD2 : \n{converted_df2.label.value_counts()}')
print(f'CIC1 : \n{converted_df3.label.value_counts()}')
print(f'CIC2 : \n{converted_df4.label.value_counts()}')


KDD1 : 
label
normal      44554
multihop        1
rootkit         1
Name: count, dtype: int64
KDD2 : 
label
normal       40406
satan         1278
ipsweep       1108
portsweep      922
nmap           465
mscan          296
saint           81
Name: count, dtype: int64
CIC1 : 
label
BENIGN                      847745
Web Attack-Brute Force         432
Web Attack-XSS                 179
Web Attack-Sql Injection         5
PortScan                         1
DoS GoldenEye                    1
Name: count, dtype: int64
CIC2 : 
label
BENIGN                      734615
DoS Hulk                     68918
DDoS                         38392
DoS GoldenEye                 3075
DoS slowloris                 1730
DoS Slowhttptest              1627
PortScan                         5
Web Attack-Sql Injection         1
Name: count, dtype: int64


In [ ]:
kdd1_count = calculate_adaptive(test_labels_kdd, y_detect_bi, 'kdd')
kdd2_count = calculate_adaptive(test_labels_kdd, y_detect_bi, 'kdd')
cic1_count = calculate_adaptive(test_labels_cic, y_detect_bi, 'cic')
cic2_count = calculate_adaptive(test_labels_cic, y_detect_bi, 'cic')


In [ ]:
from multiprocessing import Pool, cpu_count

window_size = 64
batch_size = 1
epochs = 10
n_features = X_main.shape[1]

DL_args = [window_size, batch_size, epochs]

chunk_size = 200
X = traindf.drop('label', axis=1)
y = traindf['label']

Data = TimeseriesGenerator(X, y, length=window_size, sampling_rate=1, batch_size=batch_size)


In [ ]:
def sequential_models(window_size, n_features):

    models = {
        'LSTM' : lstm(window_size, n_features)
    }

    return models

def lstm(window_size, n_features):
    model = Sequential()
    model.add(LSTM(512, input_shape=(window_size, n_features), return_sequences=True))
    model.add(Dropout(0.2))
    model.add(LSTM(512, return_sequences=True))
    model.add(Dropout(0.2))
    model.add(LSTM(256, return_sequences=True))
    model.add(Dropout(0.2))
    model.add(LSTM(128, return_sequences=True))
    model.add(Dropout(0.2))
    model.add(LSTM(64))
    model.add(Dropout(0.2))
    model.add(Dense(1, activation='sigmoid'))

    # Compile the model
    optimizer = RMSprop(learning_rate=0.0001)
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

    return model
models = sequential_models(window_size, n_features)

In [ ]:
def process_data(Data, train_index, test_index, batch_size, window_size, chunk_size=200):
    def rearrange_sequences(generator, index, batch_size, window_size):
        rearranged_data = []

        for idx in tqdm(index, desc="Processing sequences"):
            if idx >= window_size - 1:  # Ensure index has enough preceding samples for a full sequence
                sequence_end = idx + 1  # Sequence ends at the current index (inclusive)
                sequence_start = sequence_end - window_size  # Sequence starts 'window_size' samples before the end

                batch_x = generator.data[sequence_start:sequence_end]  # Extract feature sequence
                batch_y = generator.targets[idx]  # Corresponding label

                rearranged_data.append((batch_x, batch_y))

        return rearranged_data
    # Process training data
    print('Training data processing...')
    train_data = rearrange_sequences(Data, train_index, batch_size, window_size)
    gc.collect()
    
    print('Testing data processing...')
    test_data = rearrange_sequences(Data, test_index, batch_size, window_size)
    del rearrange_sequences
    gc.collect()
    return train_data, test_data
    
def separate_features_labels(data, window_size):
    features = np.array([item[0] for item in data], dtype=np.float32)  # item[0] is each sequence
    labels = np.array([item[1] for item in data], dtype=np.float32)  # item[1] is the corresponding label

    # Ensure features are reshaped to (number_of_sequences, window_size, number_of_features_per_timestep)
    features = features.reshape(-1, window_size, features.shape[-1])

    return features, labels

def training_DL(models, df, DL_args, train_test_index):
    results = {}
    X = df.drop('label', axis=1)
    y = df['label']
    
    window_size, batch_size, epochs = DL_args
    
    Data = TimeseriesGenerator(X, y, length=window_size, sampling_rate=1, batch_size=batch_size)
    print(f'Data type {type(Data)}')
    train_index, test_index = train_test_index
    print(train_index)
    print(test_index)
    print('Processing Training data...')
    
    train_data, test_data = process_data(Data, train_index, test_index, batch_size, window_size)
    gc.collect()
    X_train, y_train = separate_features_labels(train_data, window_size)
    X_test, y_test = separate_features_labels(test_data, window_size)
    return X_train, X_test, y_train, y_test

In [ ]:
Data = TimeseriesGenerator(X, y, length=window_size, sampling_rate=1, batch_size=batch_size)
print(f'Data type {type(Data)}')
train_index, test_index = train_test_index
print('Processing Training data...')

X_train, X_test, y_train, y_test = training_DL(models, traindf, DL_args, train_test_index)


In [ ]:
# window_size = 64
# batch_size = 4
# epochs = 10
X_train[0][63]

In [ ]:
train_test_index[0][0]

In [ ]:
X_train.shape

In [ ]:
y_train.shape

In [ ]:
Data[0][0]

In [ ]:
y_train[0]

In [ ]:
y

In [ ]:
sequence_start_index = train_test_index[0][1] - (window_size - 1)
sequence_start_index

In [ ]:
batch_index = sequence_start_index // batch_size
batch_index

In [ ]:
position_within_batch = sequence_start_index % batch_size
position_within_batch

In [ ]:
# Original data point
array = traindf.drop('label', axis=1).iloc[train_test_index[0][1]].to_numpy()

# Data from the generator
datagen = Data[batch_index][0][0][window_size-1]

# Comparison
equal = np.array_equal(array, datagen)
equal

In [ ]:
import glob
import os
files = glob.glob(f'kdd/Training/model/**', recursive=True)

In [ ]:
files = glob.glob(f'kdd/Training/model/**', recursive=True)
for file in files:
    if 'LSTM' in file:
        text1 = file.split('/')
        text2 = file.split('\\')
        model = text2[1]
        print(model)
        newname = file.replace('LSTM',f'{model}_LSTM_model')
        print(newname)
        os.rename(file,newname)
        print(f'Changing {file} with {newname}')

In [ ]:
files = glob.glob(f'kdd/Training/model/**', recursive=True)
for file in files:
    if 'LSTM' in file:
        print(file)

In [ ]:
from keras.models import load_model

In [ ]:
model = load_model('kdd/Training/model\land\LSTM.h5')

In [ ]:
import pandas as pd
import os
os.getcwd()


In [ ]:
df = pd.read_csv('./cic/CIC_IDS2017.csv')

In [ ]:
df

In [ ]:
df1 = pd.read_csv('./cic/train_test_folder/train_cic/train.csv')
df2 = pd.read_csv('./cic/train_test_folder/test_cic/test.csv')


In [ ]:
print(df1.shape)
print(df2.shape)

In [ ]:
1979513 + 848363

In [ ]:
df = pd.read_csv('./kdd/KDD.csv')

In [ ]:
df